# The following shows an example of detecting SRDD users from a subreddit data file. Simply change the filename in the file path to get results from other subreddit data. Alternatively, you can build a pipeline and apply it to a series of subreddit data files, however, it will take longer as each subreddit file is not small.

- before this, you need download reddit data file from academic torrent
- you should download the 10 selected subreddits related to depression
- here, teh example data file is depression_help_submission, which is one of the selected depression subreddits


In [ ]:
import sys
print("Python version:", sys.version)

Python version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import json
import zstandard as zstd
# the original file
input_file = '/content/drive/MyDrive/Reddit/depression_subreddits/depression_help_submissions.zst'
output_file = "/content/drive/MyDrive/Reddit/depression_subreddits/depression_help_submissions.json"   # this is the json file we want

i=0
# convert zst file to a json file
with open(input_file, 'rb') as compressed_file:
    dctx = zstd.ZstdDecompressor()
    with dctx.stream_reader(compressed_file) as reader:
        with open(output_file, 'w', encoding='utf-8') as json_file:
            text_stream = reader.read().decode('utf-8')  # decompress the whole file
            for line in text_stream.splitlines():       # read file line by line
                try:
                    json_obj = json.loads(line)         # each line is converted into json
                    json.dump(json_obj, json_file, ensure_ascii=False)
                    json_file.write('\n')
                    i=i+1
                except json.JSONDecodeError:
                    print(f"skip invalid JSON line: {line}")
print (i)

Mounted at /content/drive
76138


In [ ]:
#!pip install zstandard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 50.9 MB/s eta 0:00:00


In [ ]:
df = pd.read_json(output_file, lines=True)


In [ ]:
print(df.shape)

(76138, 132)


In [ ]:
import pandas as pd



# convert utc into datetime
start_timestamp = pd.Timestamp('2019-01-01 00:00:00', tz='UTC').timestamp()
end_timestamp = pd.Timestamp('2023-12-31 23:59:59', tz='UTC').timestamp()

# filter data between the observation
filtered_df = df[(df['created_utc'] >= start_timestamp) & (df['created_utc'] <= end_timestamp)]

filtered_df


,hide_score,domain,author_flair_text,created_utc,subreddit,media_embed,secure_media_embed,title,thumbnail,downs,...,top_awarded_type,url_overridden_by_dest,gallery_data,is_gallery,call_to_action,collections,author_is_blocked,previous_selftext,updated_on,_meta
14378,NaN,self.depression_help,None,1546302730,depression_help,{},{},How do I help someone with depression?,self,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14379,NaN,i.redd.it,None,1546303472,depression_help,{},{},Enjoy the good times,image,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14380,NaN,self.depression_help,None,1546306073,depression_help,{},{},Journaling - how should I go about it?,self,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14381,NaN,self.depression_help,None,1546311641,depression_help,{},{},Welp my friends aren’t responding to my texts ...,self,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14382,NaN,self.depression_help,None,1546315815,depression_help,{},{},Why is depression so hard to overcome?,default,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76133,0.0,self.depression_help,None,1704047115,depression_help,{},{},How to get motivation to go to the gym and kee...,self,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,[removed],1.704047e+09,"{'removal_type': 'automod_filtered', 'retrieve..."
76134,0.0,self.depression_help,None,1704047605,depression_help,{},{},Struggling with suicidal thoughts; I wish ther...,nsfw,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.704048e+09,{'retrieved_2nd_on': 1704177212}
76135,0.0,self.MentalHealthUK,None,1704048853,depression_help,{},{},Suicidal once more.,nsfw,0.0,...,NaN,/r/MentalHealthUK/comments/18vdve2/suicidal_on...,NaN,NaN,NaN,NaN,0.0,NaN,1.704049e+09,{'retrieved_2nd_on': 1704178494}
76136,0.0,self.depression_help,None,1704058002,depression_help,{},{},How do you reclaim a will to live?,self,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.704058e+09,{'retrieved_2nd_on': 1704187609}


In [ ]:
# only keeps columns that we need
dfposts = filtered_df[['author', 'created_utc', 'title', 'selftext']]
dfposts = dfposts[dfposts['author'] != '[deleted]']

# merge title and selftext
dfposts['content'] = dfposts['title'].fillna('') + ' ' + dfposts['selftext'].fillna('')
dfposts

,author,created_utc,title,selftext,content
14378,PsychedAlex1213,1546302730,How do I help someone with depression?,"As the title implies, I am not the one sufferi...",How do I help someone with depression? As the ...
14379,ellisstone,1546303472,Enjoy the good times,,Enjoy the good times
14380,Jirel-of-Joiry,1546306073,Journaling - how should I go about it?,I’ve had depression since I was 11. I’ve been...,Journaling - how should I go about it? I’ve ha...
14381,trefull5,1546311641,Welp my friends aren’t responding to my texts ...,My parents already left for a party so it’s me...,Welp my friends aren’t responding to my texts ...
14384,porky111,1546316743,Letter to the future me... Happy New Year ever...,,Letter to the future me... Happy New Year ever...
...,...,...,...,...,...
76133,Spiritual_Chair_7757,1704047115,How to get motivation to go to the gym and kee...,I used to love working out. I moved to a new c...,How to get motivation to go to the gym and kee...
76134,he-likes-24,1704047605,Struggling with suicidal thoughts; I wish ther...,"Hi, happy new years! I'm sorry, but i just fee...",Struggling with suicidal thoughts; I wish ther...
76135,Holiday-Plum-8054,1704048853,Suicidal once more.,,Suicidal once more.
76136,Itsprettydark,1704058002,How do you reclaim a will to live?,I’m not being hyperbolic or dramatic. Serious ...,How do you reclaim a will to live? I’m not bei...


# The following shows the procedure of how to use regular expression formulas to identify users with SRDD from posts

In [ ]:
import re
import pandas as pd

# define the re formulas
diagnosis_pattern_v1 = re.compile(
    r'(?i)\b(I was|I am|I have been|I got|I\'m|I’ve been|I’ve got)\s*(recently|just|previously|unfortunately|eventually|finally)?\s*(diagnosed with|diagnosed as having)\s*(depression|major depression|major depressive disorder|depressive disorder|clinical depression|mdd)\b'
)

diagnosis_pattern_v2 = re.compile(
    r'(?i)\b(my\s+|the\s+|a\s+|one\s+)?(doctor|therapist|psychologist|psychiatrist)\s*(recently|just|previously|unfortunately|eventually|finally)?\s*(diagnosed me with|gave me a diagnosis of)\s*(depression|major depression|major depressive disorder|depressive disorder|clinical depression|mdd)\b'
)

def extract_match_and_context(text, context_length=20):
    """
    first pattern1, if not matched, then parttern 2
    keep 20 chars before and after the SRDD statement
    """
    if pd.isna(text):
        return None, None

    # pattern_v1
    match = diagnosis_pattern_v1.search(text)
    if not match:  # if pattern_v1 not matched, try pattern_v2
        match = diagnosis_pattern_v2.search(text)

    if match:
        match_text = match.group()  # extract matched text
        start, end = match.start(), match.end()

        # extract context words (avoid exceed the index)
        start_context = max(0, start - context_length)
        end_context = min(len(text), end + context_length)
        context = text[start_context:end_context]

        return match_text, context

    return None, None


def add_match_columns(df, content_column):
    """
     add matched_text and context columns
    """
    # use apply function to run extract_match_and_context
    df[['matched_text', 'context']] = df[content_column].apply(
        lambda x: pd.Series(extract_match_and_context(x))
    )
    return df




# apply the function to dfposts
result_df = add_match_columns(dfposts, 'content')

result_df


,author,created_utc,title,selftext,content,matched_text,context
14378,PsychedAlex1213,1546302730,How do I help someone with depression?,"As the title implies, I am not the one sufferi...",How do I help someone with depression? As the ...,None,None
14379,ellisstone,1546303472,Enjoy the good times,,Enjoy the good times,None,None
14380,Jirel-of-Joiry,1546306073,Journaling - how should I go about it?,I’ve had depression since I was 11. I’ve been...,Journaling - how should I go about it? I’ve ha...,None,None
14381,trefull5,1546311641,Welp my friends aren’t responding to my texts ...,My parents already left for a party so it’s me...,Welp my friends aren’t responding to my texts ...,None,None
14384,porky111,1546316743,Letter to the future me... Happy New Year ever...,,Letter to the future me... Happy New Year ever...,None,None
...,...,...,...,...,...,...,...
76133,Spiritual_Chair_7757,1704047115,How to get motivation to go to the gym and kee...,I used to love working out. I moved to a new c...,How to get motivation to go to the gym and kee...,None,None
76134,he-likes-24,1704047605,Struggling with suicidal thoughts; I wish ther...,"Hi, happy new years! I'm sorry, but i just fee...",Struggling with suicidal thoughts; I wish ther...,None,None
76135,Holiday-Plum-8054,1704048853,Suicidal once more.,,Suicidal once more.,None,None
76136,Itsprettydark,1704058002,How do you reclaim a will to live?,I’m not being hyperbolic or dramatic. Serious ...,How do you reclaim a will to live? I’m not bei...,None,None


In [ ]:
# only keep the matched ones
match_df = result_df.dropna(subset=['matched_text']).reset_index(drop=True)
match_df

,author,created_utc,title,selftext,content,matched_text,context
0,NoMojo42,1546975345,[NSFW] No Motivation To Help Myself and Don't ...,Hi everyone. I've been a longtime lurker on th...,[NSFW] No Motivation To Help Myself and Don't ...,I was diagnosed with depression,t it off my chest.\n\nI was diagnosed with dep...
1,Shaunyantha,1547076064,My Story,I'm not even sure if this is a good idea to po...,My Story I'm not even sure if this is a good i...,I was diagnosed with depression,family. \r\nIn 2016 I was diagnosed with dep...
2,GelikJ0,1547173416,I am lying to my doctor about my depression.,I am new around here so don’t kill me if my fo...,I am lying to my doctor about my depression. I...,I was diagnosed with clinical depression,Was 16 at the time) I was diagnosed with clini...
3,Sarcastic_Emo_Fan,1547270900,I feel horrible,So yesterday I decided that I was going to tel...,I feel horrible So yesterday I decided that I ...,I was diagnosed with Depression,to tell my dad that I was diagnosed with Depre...
4,UnfoundHound,1547521299,Stuck between moving forward and killing myself,Some background information: My GF of almost 3...,Stuck between moving forward and killing mysel...,I have been diagnosed with MDD,l I did it.\n\nSo far I have been diagnosed wi...
...,...,...,...,...,...,...,...
285,TooLateMyUsernameNow,1701821972,Advice/help for feeling depressed about the pa...,"Hey everybody. So, for some background info, I...",Advice/help for feeling depressed about the pa...,I was diagnosed with MDD,"me background info, I was diagnosed with MDD w..."
286,jetpackdog,1702202873,How do I stop hating my self for my short comings,About 8 years ago when I was 15 I was diagnose...,How do I stop hating my self for my short comi...,I was diagnosed with clinical depression,s ago when I was 15 I was diagnosed with clini...
287,ohaugst,1702695757,i don't have a friend group anymore,from the past few weeks I've been thinking ab ...,i don't have a friend group anymore from the p...,i was diagnosed with depression,i was irritable but i was diagnosed with depre...
288,Cute-Ad282,1702852154,No Motivation In College Due to Having No Friends,"I'm 21, live in Massachusetts and I have been ...",No Motivation In College Due to Having No Frie...,I have been diagnosed with depression,n Massachusetts and I have been diagnosed with...


In [ ]:

# define hypothetical_pattern re formula
# remove records with hypothesis words in the context
hypothetical_pattern = re.compile(
    r'\b(if|assume|imagine|suppose|could|can|may|might|feel like|felt like|think|thought|believe|believed|hope|hoped|wish|wished|should|guess|no|never)\b',
    re.IGNORECASE  # ignore cases
)

def filter_hypothetical_rows(df, context_column):
    """
    filter out the records that have any word from the hypothetical list in the context

    """
    # apply to each line, if matched, drop them
    filtered_df = df[~df[context_column].apply(lambda x: bool(hypothetical_pattern.search(str(x))))]
    return filtered_df.reset_index(drop=True)


user_srdd_df = filter_hypothetical_rows(match_df, 'context')
user_srdd_df

,author,created_utc,title,selftext,content,matched_text,context
0,NoMojo42,1546975345,[NSFW] No Motivation To Help Myself and Don't ...,Hi everyone. I've been a longtime lurker on th...,[NSFW] No Motivation To Help Myself and Don't ...,I was diagnosed with depression,t it off my chest.\n\nI was diagnosed with dep...
1,Shaunyantha,1547076064,My Story,I'm not even sure if this is a good idea to po...,My Story I'm not even sure if this is a good i...,I was diagnosed with depression,family. \r\nIn 2016 I was diagnosed with dep...
2,GelikJ0,1547173416,I am lying to my doctor about my depression.,I am new around here so don’t kill me if my fo...,I am lying to my doctor about my depression. I...,I was diagnosed with clinical depression,Was 16 at the time) I was diagnosed with clini...
3,Sarcastic_Emo_Fan,1547270900,I feel horrible,So yesterday I decided that I was going to tel...,I feel horrible So yesterday I decided that I ...,I was diagnosed with Depression,to tell my dad that I was diagnosed with Depre...
4,UnfoundHound,1547521299,Stuck between moving forward and killing myself,Some background information: My GF of almost 3...,Stuck between moving forward and killing mysel...,I have been diagnosed with MDD,l I did it.\n\nSo far I have been diagnosed wi...
...,...,...,...,...,...,...,...
268,TooLateMyUsernameNow,1701821972,Advice/help for feeling depressed about the pa...,"Hey everybody. So, for some background info, I...",Advice/help for feeling depressed about the pa...,I was diagnosed with MDD,"me background info, I was diagnosed with MDD w..."
269,jetpackdog,1702202873,How do I stop hating my self for my short comings,About 8 years ago when I was 15 I was diagnose...,How do I stop hating my self for my short comi...,I was diagnosed with clinical depression,s ago when I was 15 I was diagnosed with clini...
270,ohaugst,1702695757,i don't have a friend group anymore,from the past few weeks I've been thinking ab ...,i don't have a friend group anymore from the p...,i was diagnosed with depression,i was irritable but i was diagnosed with depre...
271,Cute-Ad282,1702852154,No Motivation In College Due to Having No Friends,"I'm 21, live in Massachusetts and I have been ...",No Motivation In College Due to Having No Frie...,I have been diagnosed with depression,n Massachusetts and I have been diagnosed with...


- We can see that from 290 matched records, only 273 are finally kept as candidate users with SRDD, suggesting that the step of checking hypothesis in context in necessary.
- Note this is the result from **only one** depression subreddit (whose data volume is not very big)
- You can just repeat this code file on the other 9 subreddit data file, some of them (like subreddit depression) will take much more time if without GPU processor.
